# p2g

In [41]:
from g2pk2 import G2p
import pandas as pd
from tqdm import tqdm

In [42]:
g2p = G2p()
df = pd.read_csv('/data/ephemeral/home/level2-nlp-datacentric-nlp-02/data/uniform_ai_hub_bracket.csv')

seed = 456
sample = df.sample(frac=0.2, random_state=seed)

df

mecab installed


,Unnamed: 0,ID,text,target,url,date
0,0,1.601101e+06,전북도 악취배출원 기술지원단 본격 가동,6,전북일보,20210217
1,1,1.500201e+06,날뛰는 물가에 살림살이 팍팍…경북·대구 물가상승률 최고치,1,경북일보,20210506
2,2,1.400601e+06,충주에 수소융복합충전소 조성,6,충청일보,20210125
3,3,1.600801e+06,광주시 관내 1012개 어린이집 CCTV 긴급 점검,0,전남일보,20210623
4,4,1.101101e+06,안소미 오늘 15일 둘째 득남… 산모·아이 모두 건강,3,한국일보,20210215
...,...,...,...,...,...,...
19091,19091,1.101101e+06,격렬 시위에도 4연임 청신호… 네덜란드 총리 승리 비결은,4,한국일보,20210318
19092,19092,1.101101e+06,美 기내 거리두기 실종 … 전 좌석 판매 열 올린 항공사들,4,한국일보,20210503
19093,19093,1.101101e+06,골목식당 백종원 파스타 집 식전 수프에 행주 맛 나 역대급 혹평,3,한국일보,20210317
19094,19094,1.300201e+06,손흥민 남은 2경기서 차붐 넘어설까,5,강원일보,20210518


In [43]:
dropped_df = df[~(df['text'].isin(sample['text']))].drop('Unnamed: 0', axis=1)
dropped_df

,ID,text,target,url,date
1,1.500201e+06,날뛰는 물가에 살림살이 팍팍…경북·대구 물가상승률 최고치,1,경북일보,20210506
2,1.400601e+06,충주에 수소융복합충전소 조성,6,충청일보,20210125
4,1.101101e+06,안소미 오늘 15일 둘째 득남… 산모·아이 모두 건강,3,한국일보,20210215
5,1.101101e+06,서민갑부 전통시장 횟집 손님 부르는 비법은,3,한국일보,20210126
7,1.101101e+06,후쿠시마 원전 처리에 10년간 138조원 써… 앞으로도 첩첩산중,4,한국일보,20210323
...,...,...,...,...,...
19090,1.101101e+06,폼페이오 중국에 얼씬도 마 …트럼프 퇴임 선물은 28명 무더기 제재,4,한국일보,20210121
19091,1.101101e+06,격렬 시위에도 4연임 청신호… 네덜란드 총리 승리 비결은,4,한국일보,20210318
19093,1.101101e+06,골목식당 백종원 파스타 집 식전 수프에 행주 맛 나 역대급 혹평,3,한국일보,20210317
19094,1.300201e+06,손흥민 남은 2경기서 차붐 넘어설까,5,강원일보,20210518


In [44]:
for idx, data in tqdm(sample.iterrows()):
    noised_text = g2p(data['text'])
    data['text'] = noised_text
    dropped_df = pd.concat([dropped_df, pd.DataFrame([data])], ignore_index=True)

3819it [02:25, 26.22it/s]


In [53]:
dropped_df = dropped_df.drop('Unnamed: 0', axis=1)

In [54]:
dropped_df.to_csv('../data/noised_uniform_aihub_bracket.csv', index=False)

# denoise

In [47]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import pandas as pd
from tqdm import tqdm

In [55]:
data = pd.read_csv('/data/ephemeral/home/level2-nlp-datacentric-nlp-02/data/noised_uniform_aihub_bracket.csv')
data

,ID,text,target,url,date
0,1.500201e+06,날뛰는 물가에 살림살이 팍팍…경북·대구 물가상승률 최고치,1,경북일보,20210506
1,1.400601e+06,충주에 수소융복합충전소 조성,6,충청일보,20210125
2,1.101101e+06,안소미 오늘 15일 둘째 득남… 산모·아이 모두 건강,3,한국일보,20210215
3,1.101101e+06,서민갑부 전통시장 횟집 손님 부르는 비법은,3,한국일보,20210126
4,1.101101e+06,후쿠시마 원전 처리에 10년간 138조원 써… 앞으로도 첩첩산중,4,한국일보,20210323
...,...,...,...,...,...
19091,1.100201e+06,바이든 참서칸 나토…대중 강경압빡·北 CVID협쌍 촉꾸,6,국민일보,20210615
19092,1.100201e+06,퍼러비스 ESG 경영 시동,0,국민일보,20210613
19093,1.400601e+06,아이에게 머기고픈 마으므로 만듬니다,1,충청일보,20210401
19094,1.601101e+06,대한민국쑬테마방물관 스마트해진다,0,전북일보,20210208


In [56]:
tokenizer = AutoTokenizer.from_pretrained("kfkas/t5-large-korean-P2G")
model = AutoModelForSeq2SeqLM.from_pretrained("kfkas/t5-large-korean-P2G")

data = pd.read_csv('/data/ephemeral/home/level2-nlp-datacentric-nlp-02/data/noised_uniform_aihub_bracket.csv')

batch_size = 16

def denoise(batch_texts):
    inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt")
    outputs = model.generate(**inputs)
    
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

results = []

for i in tqdm(range(0, len(data), batch_size)):
    batch = data['text'][i:i+batch_size].tolist()
    batch_results = denoise(batch)
    results.extend(batch_results)

data['text'] = results


data.to_csv("/data/ephemeral/home/level2-nlp-datacentric-nlp-02/data/denoise_uniform_aihub_bracket.csv", index=False)

  0%|          | 0/1194 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 128 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
  2%|▏         | 25/1194 [04:28<3:28:53, 10.72s/it]


KeyboardInterrupt: 

In [ ]:
import pandas as pd

df1 = pd.read_csv('### original data ###')
df2 = pd.read_csv('### denoised data ###')

different = []

for i in range(len(df1)):
    if df1.loc[i, 'text'] != df2.loc[i, 'text']:
        different.append(df2.loc[i])
        
df1 = pd.concat([df1, pd.DataFrame(different)], ignore_index=True)


df1.to_csv("### save ###", index=False)